# Create animated-tiles


## Setup

### Library import


In [4]:
import sys

# Include local library paths if you have ../src/utils.py
sys.path.append("../src/")
sys.path.append("../src/animations")
sys.path.append("../src/datasets")
sys.path.append("../src/datasets/factory")

from pprint import pprint

import numpy as np
from animations.animated_tiles import AnimatedTiles
from datasets.datasets import dataset_database

# Data Acquisition
## Dataset information

In [5]:
datasets = dataset_database.datasets()
dataset = datasets.get("Meteorological Data")
layers = dataset.layers()
pprint(layers)

{'Precipitation': <datasets.datasets.Layer object at 0x7fb94aa9fe90>,
 'Temperature': <datasets.datasets.Layer object at 0x7fb94997b0b0>}


In [6]:
layer = layers["Precipitation"]
ds = layer.get_data()
ds

Loading Zarr data from gs://wbhydross_deliverables/D3-Database/02- Meteorological datasets/Rainfall-CHIRPS/WBHYDROSSD_CHIRPS_5km_Precipitation_SouthSudan_1981_2022_20240425.zarr...


<xarray.Dataset> Size: 2MB
Dimensions:               (x: 251, y: 191, month: 12)
Coordinates:
  * x                     (x) float64 2kB 23.83 23.88 23.93 ... 36.28 36.33
  * y                     (y) float64 2kB 12.67 12.62 12.57 ... 3.225 3.175
  * month                 (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    precipitation_amount  (month, y, x) float32 2MB dask.array<chunksize=(1, 24, 32), meta=np.ndarray>

In [7]:
da = ds["precipitation_amount"]
da = da.rio.set_crs("EPSG:4326")
da.attrs["crs"] = "EPSG:4326"

## Create animated-tiles

### 1-Band `GeoTIFF`

In [5]:
output_folder = "../data/processed/Precipitation/APNGs/"

**[Default colormap](https://cogeotiff.github.io/rio-tiler/colormap/#colormaps)**

You can load one of `rio-tiler`'s default colormaps from the `rio_tiler.colormap.cmap` object.

In [6]:
# Get Colormap
from rio_tiler.colormap import cmap

cm = cmap.get("YlGnBu")

**[Custom colormaps](https://cogeotiff.github.io/rio-tiler/colormap/#custom-colormaps)**

In [7]:
import matplotlib
from rio_tiler.colormap import cmap

palette = matplotlib.colors.LinearSegmentedColormap.from_list(
    "palette",
    [
        "#b9de5a",
        "#0cb627",
        "#20234b",
    ],
    256,
)

x = np.linspace(0, 1, 256)
cmap_vals = palette(x)[:, :]
cmap_uint8 = (cmap_vals * 255).astype("uint8")
cm = {idx: tuple(value) for idx, value in enumerate(cmap_uint8)}

cmap = cmap.register({"palette": cm})

**Create animated-tiles from `Xarray Dataset`**

In [7]:
animated_tiles = AnimatedTiles(
    data=da,
    output_folder=output_folder,
    min_z=5,
    max_z=5,
    color_map=cm,
    vmin=0,
    vmax=100,
    engine="xarray",
)

animated_tiles.create(time_coord="month")

Creating tiles ...


100%|██████████| 12/12 [00:46<00:00,  3.88s/it]

Creating APNGs
